# [**Bart-large-cnn**](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+was+the+first+structure+to+reach+a+height+of+300+metres.+Due+to+the+addition+of+a+broadcasting+aerial+at+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct.)

[Recursively summarize text of any length with GPT-3 (not sure if it's it)](https://www.youtube.com/watch?v=fflkFtIwQXo)

## Dependecies

In [61]:
!pip install --user -U nltk
!pip install transformers

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
You should consider upgrading via the 'D:\CVUT\DIP\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'D:\CVUT\DIP\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import nltk
from nltk import tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch

# nltk.download('punkt')

## Read data

In [69]:
df = pd.read_csv('data/summaries.csv')
# df = pd.read_csv('data/proposals_06_04.csv')
df = df[df['middleDescription'].notna() & df['juniorDescription'].notna() & df['seniorDescription'].notna()]
df.drop_duplicates(subset=["seniorDescription"], inplace=True)
df['seniorLength']  = df['seniorDescription'].str.len()

## Clean data

In [70]:
df = df[df['juniorDescription'] != 'Proposal is too long']
df[df['seniorLength'] <= 200]
df = df[df['seniorLength'] > 200]

In [71]:
display(df)

,id,title,daoId,juniorDescription,middleDescription,seniorDescription,startAt,endAt,author,createdAt,...,sum_word_density,text_word_density,sum_sent_density,text_sent_density,sum_punc_count,text_punc_count,text_stopw_count,sum_stopw_count,sum_stopw_density,text_stopw_density
0,1,[1IP-07] Integrate Balancer Boosted Pools in t...,1,The proposal is about integrating Balancer Boo...,This proposal calls for the integration of Bal...,Simple Summary. This proposal calls for the in...,1651554223000,1652159023000,0x824732d3f4eb94a20254cca9de10485ce445bb40,1657200979740,...,0.191837,0.195598,0.041667,0.049157,3,110,255,18,0.375000,0.358146
1,2,[1IP-06] Donation of 1inch DAO Treasury Funds ...,1,The 1inch DAO Treasury is proposing to donate ...,The 1inch DAO is proposing to donate 1 million...,Summary. 1inch Network was founded on core val...,1646760014000,1647364814000,0x824732d3f4eb94a20254cca9de10485ce445bb40,1657200981507,...,0.178295,0.191805,0.042553,0.053864,4,77,150,14,0.297872,0.351288
2,3,[Temperature Check] Should the Uniswap communi...,2,The Protocol Guild is a council of Ethereum pr...,"The authors are proposing that 500,000 UNI (ab...","Authors: [Trent]([link]) (PG Member), [Tim]([l...",1654176425000,1654437600000,0x4c0a466df0628fe8699051b3ac6506653191cc21,1657200982022,...,0.205036,0.178934,0.034483,0.048402,10,317,384,20,0.344828,0.350685
3,4,Aave V3 Harmony - Freeze Reserves,3,The proposal is about freezing all reserves on...,"The Aave DAO Community, through the governance...",title: Aave V3 Harmony - Freeze Reserves. stat...,1657807792000,1658138400000,0xd2362dbb5aa708bc454ce5c3f11050c016764fa6,1658239945117,...,0.193966,0.194444,0.043478,0.058394,5,34,100,17,0.369565,0.364964
4,5,Revised ARC: Add 1INCH as collateral,3,The proposal seeks to add 1inch Networks 1INCH...,1inch Network is a decentralized set of protoc...,Summary. 1inch is a network of decentralized p...,1657638000000,1657983600000,0xc290cfb8d020c0615e9c63036f4319cc41717e68,1658239945766,...,0.219048,0.192007,0.042553,0.070053,3,430,588,19,0.404255,0.314439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,805,[BIP-237] Enable USH - ETH 50/50 Gauge with 10...,12,This proposal is about adding a gauge to the n...,This proposal is about adding a gauge to the n...,[PR with Payload]([link]) Summary: Proposal to...,1680796800000,1681142400000,0x9f74662aD05840Ba35d111930501c617920dD68e,1680720307179,...,0.183575,0.190299,0.034783,0.042445,9,250,419,46,0.400000,0.355688
681,806,[BIP-239] Enable bb-a-USD gauge [Ethereum],12,This passage is describing a proposal to add a...,This passage is describing a proposal to add a...,[PR with Payload]([link]) Summary This is a pr...,1680796800000,1681142400000,0x9f74662aD05840Ba35d111930501c617920dD68e,1680720307225,...,0.203390,0.200301,0.041237,0.067416,14,79,93,38,0.391753,0.348315
682,807,[BIP-240] Enable rETH/bb-a-wETH Gauge [Arbitrum],12,This proposal is for a new pool on Arbitrum cr...,This proposal is for a new pool on Arbitrum cr...,[link] Summary: > This pool uses the Composabl...,1680796800000,1681142400000,0x512fce9B07Ce64590849115EE6B32fd40eC0f5F3,1680776108847,...,0.205882,0.184189,0.039370,0.050773,18,243,137,46,0.362205,0.302428
683,808,[BIP-241] Aave v3 Migration,12,This proposal is about transitioning from the ...,This proposal is about transitioning from the ...,[link] Motivation. > With the Aave v3 linear p...,1680796800000,1681142400000,0x512fce9B07Ce64590849115EE6B32fd40eC0f5F3,1680777004583,...,0.213793,0.180944,0.042553,0.039171,9,286,139,38,0.404255,0.320276


In [72]:
senior_lengths = df['seniorDescription'].str.len().to_numpy()
senior_lengths.sort()
print(senior_lengths[::])

[  245   272   273   290   292   322   338   356   389   410   427   437
   442   443   491   496   496   501   508   517   530   542   547   568
   570   576   584   585   585   592   595   600   600   603   608   612
   616   621   621   633   657   665   666   670   674   674   676   677
   677   679   680   680   680   683   683   684   684   685   686   686
   688   688   689   689   689   692   692   693   697   700   706   709
   724   729   732   739   742   756   764   770   775   776   780   788
   794   794   804   810   811   831   848   861   870   872   874   886
   886   899   902   928   934   959   962   966   971   974   982   987
   996  1002  1012  1041  1053  1054  1054  1058  1062  1067  1068  1084
  1085  1086  1100  1103  1107  1113  1118  1124  1139  1153  1153  1156
  1158  1159  1159  1159  1163  1167  1169  1182  1182  1185  1191  1192
  1198  1199  1206  1214  1236  1244  1248  1258  1267  1284  1292  1296
  1296  1305  1306  1322  1354  1354  1354  1354  1

## Count GPT models pricings

In [33]:
proposals_amount = 1130
GPT4_price_for_training = (sum(senior_lengths)/1000*0.03  + (685*250/1000)*0.06)
Davinci_price_finetung = (sum(senior_lengths)/1000  + (685*250/1000))*0.03
Davinci_price_usage = (sum(senior_lengths)/1000  + (685*250/1000))*0.12
ChatGPT_price_usage = (sum(senior_lengths)/1000*0.0015  + (685*250/1000)*0.002)
Curie_price_finetung = (sum(senior_lengths)/1000  + (685*250/1000))*0.003
Curie_price_usage = (sum(senior_lengths)/1000  + (685*250/1000))*0.012

print(f'GPT4 price usage: {round(GPT4_price_for_training, 1)}$')
print(f'InstructGPT Davinci (price finetung +  price usage): {round(Davinci_price_finetung, 1)}$ + {round(Davinci_price_usage, 1)}$ = {round(Davinci_price_finetung + Davinci_price_usage, 1)}$')
print(f'InstructGPT Curie (price finetung +  price usage): {round(Curie_price_finetung, 1)}$ + {round(Curie_price_usage, 1)}$ = {round(Curie_price_finetung + Curie_price_usage, 1)}$')
print(f'ChatGPT price usage: {round(ChatGPT_price_usage, 1)}$')

GPT4 price usage: 80.2$
InstructGPT Davinci (price finetung +  price usage): 75.1$ + 300.3$ = 375.3$
InstructGPT Curie (price finetung +  price usage): 7.5$ + 30.0$ = 37.5$
ChatGPT price usage: 3.8$


## Splitting text

In [12]:
checkpoint = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
print(f'Model max length {tokenizer.model_max_length}')

In [127]:
def split_text(text, limit, sep=" "):
  sentences_l0 = tokenize.sent_tokenize(text)
  sentences_l1 = []
  delimiter = '|'
  for sent in sentences_l0:
    sent_splited = sent.split(delimiter)
    if len(sent_splited) == 1:
      sentences_l1.append(sent_splited[0])
    else:
      my_list = [x+delimiter for x in sent_splited if len(sent_splited)>1]
      for i in my_list:
        sentences_l1.append(i)

  res, part, others = [], sentences_l1[0], sentences_l1[1:]
  for word in others:
    if len(sep)+len(word) > limit-len(part):
        res.append(part)
        part = word
    else:
        part += sep+word
  if part:
    res.append(part)
  return res

In [73]:
def split_text(text, limit, sep=" "):
  sentences = tokenize.sent_tokenize(text)
  length = 0
  chunk = ""
  chunks = []
  count = -1
  for sentence in sentences:
    count += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

    if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
      chunk += sentence + " " # add the sentence to the chunk
      length = combined_length # update the length counter

      # if it is the last sentence
      if count == len(sentences) - 1:
        chunks.append(chunk.strip()) # save the chunk

    else:
      chunks.append(chunk.strip()) # save the chunk

      # reset
      length = 0
      chunk = ""

      # take care of the overflow sentence
      chunk += sentence + " "
      length = len(tokenizer.tokenize(sentence))

  return chunks

In [128]:
df['seniorSplit'] = df['seniorDescription'].map(lambda x: split_text(x, limit=1024)).copy()

In [129]:
long_speech_split = (
    pd.DataFrame(df['seniorSplit'].explode())
    .stack()
    .reset_index()
    .rename(columns={0: "Sections"})
    .drop(["level_1"], axis=1)
)

In [130]:
display(long_speech_split)

,level_0,Sections
0,0,Simple Summary. This proposal calls for the in...
1,0,These pools were previously discussed with the...
2,0,In the near future we expect to see meta pools...
3,0,Support for the mentioned pools will unlock $3...
4,0,The technical specifics of this integration wi...
...,...,...
2756,684,[[1IP-14] Update 1inch DAO Voting Process to I...
2757,684,"Currently, all v1INCH, no matter where it is a..."
2758,684,New scheme = 17K votes Example 3 Address: [hex...
2759,684,Change the proposal validation strategy to req...


## Model and tokenizer

## Length checks

In [131]:
sections_list = long_speech_split['Sections'].tolist()
lens_sections_tokenized = [len(tokenizer.tokenize(c)) for c in sections_list]

In [132]:
long_speech_split['SectionsLength']  = lens_sections_tokenized
display(long_speech_split[long_speech_split['SectionsLength'] > 1024])

# display(long_speech_split[long_speech_split['SectionsLength'] > 1024]['Sections'].to_list()[2])

,level_0,Sections,SectionsLength


In [133]:
display(long_speech_split)

,level_0,Sections,SectionsLength
0,0,Simple Summary. This proposal calls for the in...,228
1,0,These pools were previously discussed with the...,212
2,0,In the near future we expect to see meta pools...,257
3,0,Support for the mentioned pools will unlock $3...,203
4,0,The technical specifics of this integration wi...,66
...,...,...,...
2756,684,[[1IP-14] Update 1inch DAO Voting Process to I...,240
2757,684,"Currently, all v1INCH, no matter where it is a...",254
2758,684,New scheme = 17K votes Example 3 Address: [hex...,262
2759,684,Change the proposal validation strategy to req...,247


In [145]:
for i in long_speech_split['Sections'].to_list()[:20]:
  print(i)
  print()

Simple Summary. This proposal calls for the integration of [Balancer Boosted Pools]([link]) into the 1inch Aggregation Protocol. Abstract. This proposal aims to grow the kinds of liquidity sources compatible with the 1inch Aggregation Protocol by integrating Balancer Boosted Pools. To fund this development, a BAL grant from Balancer DAO and Balancer DAO will be awarded to 1inch Labs upon the successful completion of the integration. Specifically, the following. 1. 1inch Labs shall integrate the Balancer Boosted Pools into the 1inch smart contracts, the 1inch dApp, and the 1inch API. 2. The integration shall be live on the Ethereum main-net no later than 3 weeks after this proposal passes the Phase-4 snapshot vote. 3. If both conditions 1 and 2 are met, Balancer DAO and Balancer DAO will transfer 20k BAL or the equivalent in stablecoins (as determined by Balancer DAO) to 1inch Labs. Motivation. Balancer Boosted Pools are a new pool type that have been gaining traction from many DeFi pro

In [140]:
lens_sections_tokenized.sort()
print(f'Amount of tokens: {lens_sections_tokenized[::-1][:10]} ...')

Amount of tokens: [596, 558, 557, 483, 445, 425, 423, 423, 423, 422] ...


In [141]:
long_speech_split_lengths = long_speech_split['Sections'].str.len().to_numpy()
long_speech_split_lengths.sort()
print(f'Amount of symbols: {long_speech_split_lengths[::-1][:10]} ... ')

Amount of symbols: [1873 1610 1354 1282 1240 1088 1078 1057 1053 1024] ... 


## Summarize long speech

In [146]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1060 (UUID: GPU-359cb4b9-ff63-1963-ff1b-3a4002faee26)


In [41]:
nlp_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

## Make summirize

In [43]:
%%time

corpus_long = list(long_speech_split['Sections'].values)
summaries_list = []

for chunk in corpus_long[0:]:
  chunk_token_len = len(tokenizer.tokenize(chunk))
  max_length = int(chunk_token_len/2) if chunk_token_len > 10 else chunk_token_len
  min_length = int(chunk_token_len/10)
  summary = nlp_summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)
  print('\nChunk:\n', chunk, '\nSummary:\n', summary, f'\nmax={max_length}, min={min_length}, len={len(chunk)}, tok={len(tokenizer.tokenize(chunk))}')
  summaries_list.append(summary)


Chunk:
 Simple Summary This proposal calls for the integration of [Balancer Boosted Pools]([link]) into the 1inch Aggregation Protocol. Abstract This proposal aims to grow the kinds of liquidity sources compatible with the 1inch Aggregation Protocol by integrating Balancer Boosted Pools. To fund this development, a BAL grant from Balancer DAO and Balancer DAO will be awarded to 1inch Labs upon the successful completion of the integration. Specifically, the following 1. 1inch Labs shall integrate the Balancer Boosted Pools into the 1inch smart contracts, the 1inch dApp, and the 1inch API. 2. The integration shall be live on the Ethereum main-net no later than 3 weeks after this proposal passes the Phase-4 snapshot vote. 3. If both conditions 1 and 2 are met, Balancer DAO and Balancer DAO will transfer 20k BAL or the equivalent in stablecoins (as determined by Balancer DAO) to 1inch Labs. Motivation Balancer Boosted Pools are a new pool type that have been gaining traction from many DeF

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Выходные данные были обрезаны до нескольких последних строк (5000).
 [{'summary_text': 'Oxytocin nominates for the position, but in a future vote we can add additional ParaSwap representatives onto our seat in the nomination table. This approval will have to be retroactive, but will still depend on the passing of this vote.'}] 
max=84, min=16, len=752, tok=169

Chunk:
 Summary This proposal asks for approval of the community to define (keep or change) the pricing approach of WBTC on Aave. The options are: 1) keep the pricing of WBTC based on the underlying BTC, same as currently on Aave v2 Ethereum and other pools OR 2) change the pricing model to use WBTC-based feeds. Context There has been extensive discussion on [[link]]([link]) about the topic, and with Aave v3 Ethereum upcoming, it is fundamental for the community to take a decision. Generally, the risk entities currently engaged with the Aave community agree that using a WBTC/USD (or ETH) feed with option 2 can bring some benefit

Token indices sequence length is longer than the specified maximum sequence length for this model (2106 > 1024). Running this sequence through the model will result in indexing errors



Chunk:
 _More information about the various Balancer Multisigs and their functions can be found in the [multisig-ops repo]([link])_ Specification As described in the payload json, the authorizer will be called to grant the roles as described. 
Summary:
 [{'summary_text': 'More information about the various Balancer Multisigs and their functions can be found in the [multisig-'}] 
max=26, min=5, len=234, tok=53


RuntimeError: ignored

In [47]:
summaries_list[0:5]

[[{'summary_text': 'Proposal calls for the integration of [Balancer Boosted Pools] into the 1inch Aggregation Protocol. To fund this development, a BAL grant from Balancer DAO and Balancers DAO will be awarded to 1inch Labs.'}],
 [{'summary_text': "Balancer seeks to fund this integration via a payment of 20,000 BAL to 1inch Labs. The integration of Balancer's BatchSwap function within the 1inch Network will also be able to be used for pools other than Boosted Pools."}],
 [{'summary_text': "In the near future we expect to see meta pools created with the BPT of Boosted Pools, an example would be a bbaUSD/WETH pool. Routing trades through such pools would include one or more extra hop through Balancer's BatchSwap function. This is contingent on the same proposal passing through the BalancerDAO."}],
 [{'summary_text': "Balancer Boosted Pools have been adapted by BeethovenX, Balancer's Friendly Fork on Fantom. Support for the pools will unlock $350mm of efficient TVL for the 1inch Aggregati

In [50]:
summaries_list[1754]

[{'summary_text': 'More information about the various Balancer Multisigs and their functions can be found in the [multisig-'}]

## Process summirized results

In [52]:
summaries = []
for summary in summaries_list:
  summaries.append(str(summary)[19:-3])

In [53]:
summaries

['Proposal calls for the integration of [Balancer Boosted Pools] into the 1inch Aggregation Protocol. To fund this development, a BAL grant from Balancer DAO and Balancers DAO will be awarded to 1inch Labs.',
 "Balancer seeks to fund this integration via a payment of 20,000 BAL to 1inch Labs. The integration of Balancer's BatchSwap function within the 1inch Network will also be able to be used for pools other than Boosted Pools.",
 "In the near future we expect to see meta pools created with the BPT of Boosted Pools, an example would be a bbaUSD/WETH pool. Routing trades through such pools would include one or more extra hop through Balancer's BatchSwap function. This is contingent on the same proposal passing through the BalancerDAO.",
 "Balancer Boosted Pools have been adapted by BeethovenX, Balancer's Friendly Fork on Fantom. Support for the pools will unlock $350mm of efficient TVL for the 1inch Aggregation Protocol.",
 'Security review and audit should be performed before this cha

In [54]:
summaries_copy = summaries.copy()

In [60]:
len(summaries_copy)

4785

In [65]:
summaries_copy[1750:1756]

['The script and data for testing the new scheme on historical data can be found here. The script also contains additional information on the distribution of rewards, profits, and payments per auction and per solver.',
 'A number of new linear pool factories and pools have been deployed to provide base tokens for LPs wishing to build on the [Core Pools concept. This BIP seeks to grant the standard administrative permissions',
 'The transaction details were generated by [this script] with inputs and outputs in [this directory] The following files can be found both linked below and in the payload PR linked above.',
 'A table that describes what each function that is being granted privileged access to allows. For each change with changes, a transaction builder named after said chain exists to apply the changes described in the tables.',
 'More information about the various Balancer Multisigs and their functions can be found in the [multisig-',
 'N']

In [57]:
for i in range(1010):
  summaries_copy += 'NaN'

In [66]:
long_speech_split["Summary"]=summaries_copy[0:2766]

In [ ]:
long_speech_split

In [ ]:
df_summirized = pd.DataFrame()
# df_summirized['id']=df['id']
df_summirized['seniorDescription']=df['seniorDescription']
df_summirized['HF_extract_summarization'] = long_speech_split[['level_0','Sections','Summary']].groupby(['level_0'])['Summary'].apply(list).str.join(' ')
df_summirized

In [70]:
df['extractiveSummarization'] = df_summirized['HF_extract_summarization']
df.drop(['seniorSplit'], axis=1)

,seniorDescription,middleDescription,juniorDescription,seniorLength,middleLength,juniorLength,sum_sent_count,text_sent_count,sum_word_count,text_word_count,...,text_word_density,sum_sent_density,text_sent_density,sum_punc_count,text_punc_count,text_stopw_count,sum_stopw_count,sum_stopw_density,text_stopw_density,extractiveSummarization
0,Simple Summary This proposal calls for the int...,This proposal calls for the integration of Bal...,The proposal is about integrating Balancer Boo...,4322,320,290,2,28,47,715,...,0.196861,0.041667,0.039106,3,107,259,18,0.375000,0.361732,Proposal calls for the integration of [Balance...
1,Summary 1inch Network was founded on core valu...,The 1inch DAO is proposing to donate 1 million...,The 1inch DAO Treasury is proposing to donate ...,2629,365,302,2,18,46,429,...,0.193330,0.042553,0.041860,4,75,153,14,0.297872,0.355814,1inch Network was founded on core values of de...
2,"Authors: [Trent]([link]) (PG Member), [Tim]([l...","The authors are proposing that 500,000 UNI (ab...",The Protocol Guild is a council of Ethereum pr...,7165,504,330,2,47,57,1100,...,0.179915,0.034483,0.042688,10,317,390,20,0.344828,0.354223,Protocol Guild Pilot is a vested split contrac...
3,title: Aave V3 Harmony - Freeze Reserves statu...,"The Aave DAO Community, through the governance...",The proposal is about freezing all reserves on...,1669,879,275,2,9,45,274,...,0.195994,0.043478,0.032727,5,28,101,17,0.369565,0.367273,This proposal calls for the Aave Guardian to f...
4,Summary 1inch is a network of decentralized pr...,1inch Network is a decentralized set of protoc...,The proposal seeks to add 1inch Network's 1INC...,11517,745,255,2,91,47,1874,...,0.193216,0.041667,0.048533,4,395,593,20,0.416667,0.316267,1inch is a network of decentralized protocols ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,[PR with Payload]([link]) Summary: Proposal to...,This proposal is about adding a gauge to the n...,This proposal is about adding a gauge to the n...,7333,731,731,4,48,114,1184,...,0.191276,0.034783,0.040506,9,255,426,46,0.400000,0.359494,N a N N a N N a
681,[PR with Payload]([link]) Summary This is a pr...,This passage is describing a proposal to add a...,This passage is describing a proposal to add a...,1574,561,561,4,20,96,266,...,0.200000,0.041237,0.074906,14,81,93,38,0.391753,0.348315,N N
682,[link] Summary: > This pool uses the Composabl...,This proposal is for a new pool on Arbitrum cr...,This proposal is for a new pool on Arbitrum cr...,2910,735,735,5,26,127,457,...,0.185621,0.039062,0.056769,19,251,142,47,0.367188,0.310044,a N N a
683,[link] Motivation > With the Aave v3 linear po...,This proposal is about transitioning from the ...,This proposal is about transitioning from the ...,2790,523,523,4,14,93,434,...,0.181514,0.042553,0.032184,9,284,140,38,0.404255,0.321839,N N a


In [71]:
df.to_csv('out.csv', index=False)

In [ ]:
files.download('export_dataframe.csv')

---

#**[Long Document Summarization using Hugging Face Transformers | BERT, BART, DistilBART, T5, Pegasus](https://www.youtube.com/watch?v=78KjuKYiF6s)**